<a href="https://colab.research.google.com/github/opsifiz/AIB5_Emotion-Classification/blob/main/archive/create_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('drive/MyDrive/cleaned_dataset.csv', index_col=False)

In [11]:
df

,text,status,from
0,oh my gosh,Anxiety,df1
1,trouble sleeping confused mind restless heart ...,Anxiety,df1
2,all wrong back off dear forward doubt stay in ...,Anxiety,df1
3,i have shifted my focus to something else but ...,Anxiety,df1
4,i am restless and restless it is been a month ...,Anxiety,df1
...,...,...,...
1093453,where can you go to get help for someone menta...,mentalillness,df3
1093454,i am rooster illusion ama,mentalillness,df3
1093455,i look again to make sure so one is near me an...,mentalillness,df3
1093456,nami peertopeer is a unique experiential learn...,mentalillness,df3


# Create Dataset for Model #1

In [14]:
temp = df[df['status'].isin(['BPD', 'Anxiety', 'depression', 'mentalillness', 'bipolar', 'schizophrenia', 'Normal'])]
temp = temp.groupby('status').head(15000)

temp

,text,status,from
0,oh my gosh,Anxiety,df1
1,trouble sleeping confused mind restless heart ...,Anxiety,df1
2,all wrong back off dear forward doubt stay in ...,Anxiety,df1
3,i have shifted my focus to something else but ...,Anxiety,df1
4,i am restless and restless it is been a month ...,Anxiety,df1
...,...,...,...
1039331,low testosterone after discontinuing rispredon...,schizophrenia,df3
1039332,how did you finally accept your diagnosis i am...,schizophrenia,df3
1039333,constantly feel like i am in a competition wit...,schizophrenia,df3
1039334,has anyone switched over to an entirely differ...,schizophrenia,df3


In [ ]:
temp.to_csv('drive/MyDrive/train_dataset.csv', index=False)

#Create Dataset for Model #2

In [30]:
temp = df[df['status'].isin(['Anxiety', 'BPD', 'Normal', 'Stress', 'Suicidal', 'bipolar', 'depression', 'mentalillness', 'schizophrenia'])]
temp = temp.groupby('status').head(10000)
temp.drop(columns=['from'], inplace=True)

temp

,text,status
0,oh my gosh,Anxiety
1,trouble sleeping confused mind restless heart ...,Anxiety
2,all wrong back off dear forward doubt stay in ...,Anxiety
3,i have shifted my focus to something else but ...,Anxiety
4,i am restless and restless it is been a month ...,Anxiety
...,...,...
1034331,stuff my voices tell me they say that they hat...,schizophrenia
1034332,bipolar and schizophrenia in remission after w...,schizophrenia
1034333,nonpharmaceutical therapies hi i am someone wh...,schizophrenia
1034334,got a job at baskin robbins what is your exper...,schizophrenia


In [31]:
temp['status'].value_counts()

,count
status,
Anxiety,10000
Normal,10000
depression,10000
Suicidal,10000
bipolar,10000
schizophrenia,10000
BPD,10000
mentalillness,10000
Stress,2272


In [32]:
temp = temp.reset_index(drop=True)
temp.fillna('-', inplace=True)

In [7]:
n = len(temp)

n

90000

In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import LabelEncoder
import pandas as pd

# สมมุติว่า df มีคอลัมน์ 'text' และ 'status'
# df = pd.read_csv("your_file.csv")

# 1. แปลง label (status) เป็นตัวเลข
le = LabelEncoder()
temp['label'] = le.fit_transform(temp['status'])

# 2. Vectorize ข้อความ
vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(temp['text'])
y = temp['label']

# 3. ใช้ SMOTE เพื่อ balance class ทั้งหมด
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# 4. แปลงกลับเป็น DataFrame
resampled_df = pd.DataFrame(X_resampled.toarray(), columns=vectorizer.get_feature_names_out())
resampled_df['label'] = y_resampled
resampled_df['status'] = le.inverse_transform(y_resampled)

# ดูจำนวนต่อ class หลัง SMOTE
print(resampled_df['status'].value_counts())


status
Anxiety          10000
Normal           10000
depression       10000
Suicidal         10000
Stress           10000
bipolar          10000
BPD              10000
schizophrenia    10000
mentalillness    10000
Name: count, dtype: int64


In [37]:
resampled_df['status'].value_counts()

,count
status,
Anxiety,10000
Normal,10000
depression,10000
Suicidal,10000
Stress,10000
bipolar,10000
BPD,10000
schizophrenia,10000
mentalillness,10000


In [38]:
temp.to_csv('drive/MyDrive/train_dataset2.csv', index=False)